# RSA Algorithm

This is my implementation of the RSA algorithm. I didn't see any python code for it so I made some. Watch the Khan Academy series for an explanation of the algorithm. Or this video is super helpful as well.
If there is something wrong with it please message me.

In [130]:
# How to encrypt a message (general form)
# m^e mod n = c

# How to decrypt a message (general form)
# c^d mod n = m

message = 69  # we cannot let the teacher see this message

In [131]:
# Encrypt a message 
def encrypt(m, e, n):
    return (m**e) % n

In [132]:
# A function which gets the greatest common divisor
# of 2 numbers (a,b)
def getGCD(a, b):
    return a if (b==0) else getGCD(b,a%b)

In [133]:
import random

# Calculates the e value for the public key
# the definition of e is that e must share
# no factors with phi except 1 and must be
# in the range 2 < e < phi(n)
def calculateE(phi):
    testVar = random.randint(3, phi)
    if (getGCD(phi, testVar) == 1):
        return testVar
    else:
        return calculateE(phi)

In [134]:
# Calculates the value of d (private key)
# My implementation of the Extended Euclidean Algorithm
def calculateD(phi, e):
    def calculateD_helper(a,b,c,d,prev_c=None):
        if(b==1):
            return d
        else:
            mult = int(a/b)
            new_d = c - (d*mult) if (c > d) else (c-d) + prev_c
            return calculateD_helper(b, a-(b*mult),d,new_d,prev_c=c)
    
    return calculateD_helper(phi, e, phi, 1)

In [135]:
# Decrypt an encrypted message
def decrypt(c, d, n):
    return (c**d) % n

In [136]:
# PUTTING IT ALL TOGETHER #

# The Secret Sauce #
# p and q are supposed to be strong primes
# but in this case we don't really care
p = 61
q = 53
phi = (p-1)*(q-1)
####################

### Public Key ###
# Anyone can see n and e, they use these numbers to encrypt their message
# All other variables (p,q,phi,d) are not visible to anyone but the person that recieves the message
# Commonly referred to as the padlock
n = p * q
e = 17 #calculateE(phi)
##################

### Private Key ###
# Only we can calculate d because we have e and phi
d = calculateD(phi, e)
###################

In [137]:
# !IMPORTANT! #
# In this case it can matter, message < n #
message = 69
print(message)

69


In [138]:
# Encrypt the message 8008135
encrypted_message = encrypt(message, e, n)
print(encrypted_message)

28


In [139]:
# Decrypt an encrypted message with d
decrypted_message = decrypt(encrypted_message, d, n)
print(decrypted_message)
# TADA! #

69


In [140]:
# I am not sure how to properly encrypt a sentence or something but here is a bad way
# I'll look into this in the future...
# encrypted_message = [encrypt(x, e, n) for x in map(ord, message)]
# decrypted_message = "".join([chr(decrypt(x, phi, e, n)) for x in encrypted_message])

In [141]:
import math

# How to solve RSA algorithm
# Find prime factors of n... not easy for large n
def getPrimeFactors(n):
    # Very simple check if a number is prime
    def isprime(x):
        if x != 2 and x % 2 == 0:
            return False
        for i in list(range(3, int(math.sqrt(x)+1)))[::2]:
            if x % i == 0:
                return False
        else:
            return True
    
    # get list of all factors
    # find a pair of primes
    for x in list(range(2,int(n/2)+1)):
        if (n % x == 0):
            if (isprime(x) and isprime(n/x)):
                return x, n/x

In [142]:
# Get the prime factors of n, don't try this on large n
getPrimeFactors(n)

(53, 61.0)

In [143]:
# Hooray we found p and q! #

## The Seive of Eratosthenes
#### Steps
* Generate a list of numbers from 1 to some number (n)
* Initially, all numbers in the list are unmarked
* Starting at 2 and then for every element in the list mark every multiple of every number you iterate over (marking it twice does not negate it)
* Do this for every number up to and including the square root of n
* All unmarked numbers [2,n] are prime numbers

#### Example of completed sieve for numbers up to 100
<img src="files/data/sieve_100.png">

In [144]:
# Get list of prime numbers up to n 
def sieveOfEratosthenes(n, numbers=[], count=0):
    if numbers == []:
        numbers = list(range(2,n))  # math.floor(math.sqrt(n))
    if math.floor(math.sqrt(n)) < numbers[count]:
        return numbers[:count+1] + list(filter(lambda x: x % numbers[count] != 0,numbers[count:]))
    else:
        new_nums = numbers[:count+1] + list(filter(lambda x: x % numbers[count] != 0,numbers[count:]))
        return sieveOfEratosthenes(n, new_nums, count+1)    

In [145]:
test = sieveOfEratosthenes(100)
print("Number of prime numbers below number: " + str(len(test)))
print(test)

Number of prime numbers below number: 25
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


In [146]:
# Given a number to test (n) this will give you
# the fastest time to make sure you have a prime number
# the prime list should be all primes from 2 to n 
# (not good unless you can store all of that and may use it multiple times)
def isPrimeFromList(n, prime_list):
    for prime in prime_list:
        if n % prime == 0:
            return False
    return True

num = 101
isPrimeFromList(num, sieveOfEratosthenes(int(math.sqrt(num))))

True

## Fast Modular Exponentiation

In [147]:
from functools import reduce  # scala plz

# used for understanding, just use the unbounded ** python operator
def fastModularExponentiation(base, exponent, modulus):
    # form of base^(exponent) mod modulus
    powers_of_2 = []
    for ix, bit in enumerate(list(reversed(bin(exponent)[2:]))):
        if bit == '1':
            powers_of_2.append((base**(2**(ix)))%modulus)
    
    return reduce(lambda x,y: x*y, powers_of_2, 1) % modulus


In [148]:
# (200^20) % 17
fastModularExponentiation(200,20,17)

1

## Probabilistic Primes
#### (Will put up more thorough explanation soon. Maybe some pictures too!)

This is a function that will tell you if a number (p) is a prime number with a certain chance of error (ex. 1/(2^20)). It works by checking if a number shares a common denominator with a random number from 2 to p (in which case the number is composite) and then if it does not share a common denominator > 1 with a, we use fermat's little theorem which states that a base (a)^(p-1) mod p (is congruent to) 1. However this is not ALWAYS true it is possible for 1/2 of the numbers from 2 to p to be fermat liars (values of a that don't satisfy this theorem). So our error is 1/(2^t) where t is the number of different random bases we use. The function also creates a set of numbers that have already been tested because you don't want to use the same base to test against because that will mess with your error numbers.

In [149]:
from random import randint

## The beauty of this is that the time it takes to decide if a number is a prime
## scales with the error that you supply rather than the magnitude of the number you provide.
## (Except that you still have to raise a base a to the power of that number)
def fermatPrimalityTest(p, error):
    # error = 1/(2**n_tests)
    if error <= 0 or error >= 1:
        raise Exception("Error must be 0 < error < 1.")
        
    n_tests = math.ceil(math.log(1/error)/math.log(2))
    tested_numbers = set()
    for _ in range(0, n_tests):
        a = randint(2,p)
        while a in tested_numbers:
            a = randint(2,p)
        if getGCD(p,a) != 1:
            return False
        if (a**(p-1)) % p != 1:
            return False
    return True

In [150]:
fermatPrimalityTest(5613,1/(2^20))

False

In [128]:
## More to come?